In [1]:
pip install ipynb --upgrade

Requirement already up-to-date: ipynb in c:\users\alex-\anaconda3\lib\site-packages (0.5.1)Note: you may need to restart the kernel to use updated packages.



In [2]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [3]:
os.getcwd()


'C:\\Users\\Alex-\\Desktop\\Beslutsstödssystem\\models-decision-system\\MockPipeline'

In [4]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [5]:
settings = load_yaml('mock_settings.yaml')

In [6]:
settings['ensemble']['models'][0]

{'linreg': {'split': {'train': 0.8, 'test': 0.2}}}

In [7]:
dataframe = processing.create_dataframe(settings)

In [8]:
dataset = features.add(dataframe, settings['features'])

In [9]:
dataset

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409
...,...,...,...,...
2019-12-25,1.10944,0.00301,54.215370,1.11012
2019-12-26,1.11012,0.00089,50.207241,1.11713
2019-12-27,1.11713,0.00388,49.232845,1.11813


In [10]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [11]:
linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [12]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [13]:
settings['ensemble']['models'][1]['lstm']['layers']

[{'lstm': {'value': 120, 'activation': 'relu'}},
 {'dropout': {'value': 0.15}},
 {'dense': {'value': 50, 'activation': 'relu'}},
 {'dense': {'value': 1}}]

In [14]:
lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

Epoch 1/20
233/233 [==============================] - 1s 5ms/step - loss: 0.0264 - val_loss: 0.0057
Epoch 2/20
233/233 [==============================] - 1s 4ms/step - loss: 0.0058 - val_loss: 6.1926e-04
Epoch 3/20
233/233 [==============================] - 1s 4ms/step - loss: 0.0039 - val_loss: 0.0016
Epoch 4/20
233/233 [==============================] - 1s 4ms/step - loss: 0.0032 - val_loss: 0.0011
Epoch 5/20
233/233 [==============================] - 1s 4ms/step - loss: 0.0028 - val_loss: 5.8089e-04
Epoch 6/20
233/233 [==============================] - 1s 3ms/step - loss: 0.0024 - val_loss: 5.4235e-04
Epoch 7/20
233/233 [==============================] - 1s 3ms/step - loss: 0.0020 - val_loss: 5.0719e-04
Epoch 8/20
233/233 [==============================] - 1s 3ms/step - loss: 0.0019 - val_loss: 6.2374e-04
Epoch 9/20
233/233 [==============================] - 1s 3ms/step - loss: 0.0017 - val_loss: 0.0028
Epoch 10/20
233/233 [==============================] - 1s 3ms/step - loss: 0.001

In [15]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary

batch_size, timesteps, input_dim = None, 20, 1


def get_x_y(size=1000):
    import numpy as np
    pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    x_train = np.zeros(shape=(size, timesteps, 1))
    y_train = np.zeros(shape=(size, 1))
    x_train[pos_indices, 0] = 1.0
    y_train[pos_indices, 0] = 1.0
    return x_train, y_train


i = Input(batch_shape=(batch_size, timesteps, input_dim))

o = TCN(return_sequences=False)(i)  # The TCN layers are here.
o = Dropout(0.15)(o)
o = Dense(50)(o)
o = Dense(1)(o)

m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='mse')

tcn_full_summary(m, expand_residual_blocks=False)

x, y = get_x_y()
m.fit(x, y, epochs=10, validation_split=0.2)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 20, 64), (None, 2 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 20, 64), (None, 2

In [16]:
m.predict(x)

array([[9.0483017e-04],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.0480968e-04],
       [9.0480968e-04],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.8594171e-01],
       [9.8594171e-01],
       [9.0483017e-04],
       [9.0483017e-04],
       [9.8594171e-01],
       [9.859417

In [17]:
def temporal_convolutional_network(data, settings):

    #  VARIABLES
    batch_size = settings['batch']
    timesteps = 3
    input_dim = 1#len(data['train']['features'])
    x_train = data['train']['features']
    y_train = data['train']['labels']
    x_validation = data['validation']['features']
    y_validation = data['validation']['labels']
    x_test = data['test']['features']
    scaler = data['scaler']

    model_input = Input(batch_shape=(batch_size, timesteps, input_dim))

    model_output = TCN(return_sequences=False)(model_input)  # The TCN layers are here.
    model_output = Dropout(0.15)(model_output)
    model_output = Dense(50)(model_output)
    model_output = Dense(1)(model_output)

    model = Model(inputs=[model_input], outputs=[model_output])
    model.compile(optimizer='adam', loss='mse')

    model.fit(x_train, y_train, epochs=settings['epochs'], validation_data=(x_validation, y_validation))
    #  https://keras.io/api/models/model_training_apis/#fit-method

    predictions = model.predict(x_test)

    return {
        'model': model,
        'predictions': predictions
    }

In [18]:
settings['ensemble']['models'][3]['tcn']

{'split': {'train': 0.6, 'test': 0.2, 'validation': 0.2},
 'layers': [{'tcn': {'value': 10, 'activation': 'relu'}},
  {'dropout': {'value': 0.15}},
  {'dense': {'value': 50, 'activation': 'relu'}},
  {'dense': {'value': 1}}],
 'epochs': 10,
 'batch': 4,
 'validation': 40,
 'loss': 'mean_squared_error',
 'optimizer': 'adam'}

In [19]:
temporal_convolutional_network(data_lstm, settings['ensemble']['models'][3]['tcn'])

Epoch 1/10


InvalidArgumentError:  Incompatible shapes: [32,3,64] vs. [4,1,64]
	 [[node functional_3/tcn_1/residual_block_0/spatial_dropout1d_12/dropout/Mul_1 (defined at C:\Users\Alex-\anaconda3\lib\site-packages\tcn\tcn.py:145) ]] [Op:__inference_train_function_18469]

Function call stack:
train_function


In [20]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn import preprocessing
from sklearn import utils

In [21]:
df = pd.read_csv("../MockData.csv")
df_dec = pd.read_csv("../decision.csv")

In [22]:
df_dec = df_dec.set_index("DateTime")
df = df.set_index("DateTime")

In [23]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split

In [24]:
df

,Close,RoC,S%D,PSY12,ASY5
DateTime,,,,,
2010-01-27,0.817467,0.279074,0.239967,0.272727,0.452398
2010-01-28,0.802634,0.256945,0.173515,0.272727,0.376690
2010-01-29,0.779823,0.260848,0.136936,0.272727,0.291014
2010-01-31,0.782677,0.276472,0.108876,0.363636,0.304610
2010-02-01,0.793195,0.281944,0.078999,0.363636,0.385942
...,...,...,...,...,...
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453
2019-12-27,0.175454,0.565530,0.421044,0.818182,0.586624
2019-12-29,0.177701,0.567630,0.451166,0.818182,0.585169


In [25]:
len(df)

3106

In [26]:
df

,Close,RoC,S%D,PSY12,ASY5
DateTime,,,,,
2010-01-27,0.817467,0.279074,0.239967,0.272727,0.452398
2010-01-28,0.802634,0.256945,0.173515,0.272727,0.376690
2010-01-29,0.779823,0.260848,0.136936,0.272727,0.291014
2010-01-31,0.782677,0.276472,0.108876,0.363636,0.304610
2010-02-01,0.793195,0.281944,0.078999,0.363636,0.385942
...,...,...,...,...,...
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453
2019-12-27,0.175454,0.565530,0.421044,0.818182,0.586624
2019-12-29,0.177701,0.567630,0.451166,0.818182,0.585169


In [27]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day
DateTime,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold
2018-05-23,0.333239,0.299195,NaN,NaN,Hold
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy
...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold


In [78]:
    forecast_days = 1
    #df['Label'] = df['Close'].shift(-forecast_days)
    df_dec.dropna()
    data = df_dec
    data.dropna(how="any", inplace=True)
    x = data[["Forecast","Actual", "f_momentum","a_momentum"]].to_numpy()
    y = data["Invest next day"].to_numpy()

X = data.iloc[:, :4].values
    y = data.iloc[:, -1].values
    
    print(X)

In [79]:
tss = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tss.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)

In [80]:
from sklearn.preprocessing import MinMaxScaler

In [81]:
scaler = MinMaxScaler(feature_range=(0,1))

x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

In [82]:
y_train[:50]

array(['Sell', 'Hold', 'Buy', 'Sell', 'Hold', 'Hold', 'Buy', 'Sell',
       'Hold', 'Hold', 'Hold', 'Buy', 'Hold', 'Hold', 'Hold', 'Sell',
       'Buy', 'Hold', 'Sell', 'Hold', 'Hold', 'Buy', 'Sell', 'Hold',
       'Hold', 'Hold', 'Buy', 'Hold', 'Hold', 'Sell', 'Hold', 'Buy',
       'Hold', 'Sell', 'Hold', 'Hold', 'Hold', 'Buy', 'Hold', 'Hold',
       'Hold', 'Sell', 'Hold', 'Hold', 'Buy', 'Hold', 'Hold', 'Sell',
       'Hold', 'Buy'], dtype=object)

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

In [83]:
svc = LinearSVC()
svc.fit(x_train, y_train)
predict = svc.predict(x_test)

print('Accuracy of Linear SVC classifier on training set: {:.2f}'
 .format(svc.score(x_train, y_train)))

predict

Accuracy of Linear SVC classifier on training set: 0.64


array(['Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold'], dtype=object)

In [94]:
test_data = {
    'train':{
        'features': x_train,
        'labels': y_train
    },
    'test':{
        'features': x_test,
        'labels': y_test
    },
    'scaler': scaler
}

In [95]:
def linear_support_vector_classifier(data, settings):
    """Creates a Linear Support Vector Classifier (Linear SVC) and predictions.

    Args:
        data: pandas.DataFrame.
        settings: Dictionary object containing settings parameters.
    Returns:
        A dictionary containing the Linear SVC model and predictions.
    """
    
     #  VARIABLES
    x_train = data['train']['features']
    y_train = data['train']['labels']
    x_test = data['test']['features']
    scaler = data['scaler']

    # INSTANTIATE MODEL
    model = LinearSVC()
    
    # CREATE PREDICTIONS USING TRAIN DATA
    model.fit(x_train, y_train)
    
    # PREDICTIONS
    predictions = model.predict(x_test)

    return {
        'model': model,
        'predictions': predictions
    }

In [97]:
linear_support_vector_classifier(test_data, settings['ensemble']['models'][4]['svc'])

{'model': LinearSVC(),
 'predictions': array(['Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
        'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
        'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
        'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
        'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
        'Hold', 'Hold', 'Hold', 'Hold', 'Hold'], dtype=object)}